In [1]:
import ee, os
import geemap
# ee.Initialize()
Map = geemap.Map() 
Map = geemap.Map(center=(30, 115), zoom=7)
Map

Map(center=[30, 115], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

In [2]:
h = 110
v = 25
#Year
IniYear = 2020

LowLat = v
UpLat = v+5
LeftLon = h
RightLon = h+5

#Regions
region = ee.Geometry.Rectangle(LeftLon,LowLat,RightLon,UpLat);
Map.addLayer(region)

dem = ee.Image('USGS/SRTMGL1_003')
#Set region and starting and ending time
start_date = str(IniYear) + '-01-01';
end_date = str(IniYear) + '-12-31';
print(start_date)
print(end_date)

2020-01-01
2020-12-31


In [3]:
def cloudMaskL457(image):
    qa = image.select('pixel_qa')
  # If the cloud bit (5) is set and the cloud confidence (7) is high
  # or the cloud shadow bit is set (3), then it's a bad pixel.
    cloud = qa.bitwiseAnd(1 << 5) \
                  .And(qa.bitwiseAnd(1 << 7)) \
                  .Or(qa.bitwiseAnd(1 << 3))
  # Remove edge pixels that don't occur in all bands
    mask2 = image.mask().reduce(ee.Reducer.min())
    return image.updateMask(cloud.Not()).updateMask(mask2).divide(10000).copyProperties(image, ["system:time_start",'system:id'])

def maskS2clouds(image):   # This function was used to mask the clouds for sentinel-2
    qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))

  # Return the masked and scaled data, without the QA bands.
    return image.updateMask(mask).divide(10000).select("B.*").copyProperties(image, ["system:time_start",'system:id'])

def maskL8sr(image):
  # Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    # Get the pixel QA band.
    qa = image.select('pixel_qa')
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                 .And(qa.bitwiseAnd(cloudsBitMask).eq(0)).divide(10000)
    return image.updateMask(mask).copyProperties(image, ["system:time_start",'system:id'])

In [4]:
#function: Terrain.hillShadow
def filterBadObs(image):
    #selecet SOLAR_AZIMUTH_ANGLE
    azimuth = ee.Number(image.get('SOLAR_AZIMUTH_ANGLE'))
    #selecet SOLAR_ZENITH_ANGLE
    zenith = ee.Number(image.get('SOLAR_ZENITH_ANGLE'))
    #calculate Terrain shadow
    shadow_original = ee.Terrain.hillShadow(dem,azimuth,zenith,100) 
    shadow = shadow_original.focal_min(1,"square","pixels",1)
    shadow_band0  = ee.Image(0).where(shadow.eq(1),1)
    #return masked imgage
    return image.updateMask(shadow_band0)


In [ ]:
def f_EVI(img):
    EVI = img.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
          'NIR': img.select('B3').multiply(0.0001),
          'RED': img.select('B2').multiply(0.0001),
          'BLUE':img.select('B1').multiply(0.0001)}).rename('EVI')
    return EVI

def addVIs(img):
    NDVI = img.normalizedDifference(['B4','B3']).rename('NDVI')
    mNDWI = img.normalizedDifference(['B2','B5']).rename('mNDWI')
    LSWI = img.normalizedDifference(['B4','B5']).rename('LSWI')
    EVI = f_EVI(img)
    return img.addBands(NDVI).addBands(mNDWI).addBands(LSWI).addBands(EVI)

In [ ]:
#get Landsat8 collction
collection8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
    .filterBounds(region)\
    .filterDate(start_date, end_date)\
    .map(filterBadObs)\
    .map(maskL8sr)\
    .select( #Rename LC8 bands same as L5/7
      ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'pixel_qa']
      ,['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'])\
      .map(addVIs)
print('Landsat8')
print(collection8.size().getInfo())

#get Landsat7 collction
collection7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')\
    .filterBounds(region)\
    .filterDate(start_date, end_date)\
    .map(filterBadObs)\
    .map(cloudMaskL457)\
    .select( 
      ['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'])\
     .map(addVIs)
print('Landsat7')
print(collection7.size().getInfo())

#get Landsat5 collction
collection5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR')\
    .filterBounds(region)\
    .filterDate(start_date, end_date)\
    .map(filterBadObs)\
    .map(cloudMaskL457)\
    .select( 
      ['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'])\
     .map(addVIs)
print('Landsat5')
print(collection5.size().getInfo())


#Merge to a single collection
col1 = ee.ImageCollection(collection5.merge(collection7));
collection = ee.ImageCollection(col1.merge(collection8));
print(collection.size().getInfo())

In [ ]:
#water detection
def Water(img):
    return img.select('mNDWI').gt(img.select('EVI'))\
    .Or(img.select('mNDWI').gt(img.select('NDVI')))\
    .And(img.select('EVI').lt(0.1))

#Non-Water detection
def NonWater(image):
    return image.select('mNDWI').lte(image.select('EVI'))\
    .And(image.select('mNDWI').lte(image.select('NDVI')))\
    .Or(image.select('EVI').gte(0.1))

#fill zero in front of a number.
def PrefixInteger(num, n):
    return (Array(n).join(0) + num).slice(-n)



In [ ]:
#water frequency and quality band (number of good observations)
total_Pos = collection.map(Water).sum()
total_Neg = collection.map(NonWater).sum()

In [ ]:
#freqName = 'Freq_usgs578_' + str(IniYear) + '_' + str(PrefixInteger(LeftLon,4)) + '_' + str(PrefixInteger(LowLat,3))
freqName = 'Freq_usgs578_'+str(IniYear)+'_'+str(LeftLon).zfill(4)+'_'+str(LowLat).zfill(3)

print(freqName)

In [ ]:
# flooding_freq = (total_Pos.divide(total_Pos.add(total_Neg))).multiply(254).add(ee.Image(1)).rename("flooding_freq")
flooding_freq = ee.Image(0).expression('((Pos)/(Pos + Neg))*254+1',
                        {
                            'Pos':total_Pos,
                            'Neg':total_Neg
                        }).rename("flooding_freq")

vis = {
    'min':0,
    'max':255
}
Map.addLayer(flooding_freq.toUint8().clip(region),vis,'flooding_freq')

In [ ]:
freq_image = flooding_freq.toUint8().clip(region)
freq_image.getInfo()

In [5]:
collection8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
    .filterBounds(region)\
    .filterDate(start_date, end_date)\
    .select( #Rename LC8 bands same as L5/7
      ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'pixel_qa']
      ,['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'])
print('Landsat8')
print(collection8.size().getInfo())

Landsat8
257


In [7]:
Map.addLayer(collection8.mosaic().toUint8().clip(region), {}, 'image')
Map

Map(bottom=14063.0, center=[27.6543381066919, 112.61425887007147], controls=(WidgetControl(options=['position'…

In [8]:
image = collection8.mosaic().toUint8().clip(region)

In [10]:
from ee_export import ee_export
sv_file = 'E:/Work/TEMP'
ee_export(region=region, image=image, outfile=sv_file, scale=30, sep=0.5)

分割成110份, 开始下载:
Generating URL ...
Please wait ...
An error occurred while downloading.
Generating URL ...
An error occurred while downloading.
('Cannot connect to proxy.', RemoteDisconnected('Remote end closed connection without response'))
Generating URL ...
Please wait ...
An error occurred while downloading.


UnboundLocalError: local variable 'r' referenced before assignment

In [54]:
from Monarch.user_gee import clip_dow_merge
sv_file = 'E:/Work/TEMP'
while not os.path.exists(sv_file+'.tif'):
    clip_dow_merge(region, image, sv_file, 100, sep=0.5)

分割成12份, 开始下载:
Generating URL ...
Please wait ...
An error occurred while downloading.
Generating URL ...
Please wait ...
An error occurred while downloading.
Generating URL ...
Please wait ...
An error occurred while downloading.
Generating URL ...
Please wait ...
An error occurred while downloading.


UnboundLocalError: local variable 'r' referenced before assignment

In [57]:
geemap.ee_export_image(image, sv_file+'.tif', 300, region=region)

Generating URL ...
An error occurred while downloading.
Pixel grid dimensions (18554x18642) must be less than or equal to 10000.


In [62]:
region.getInfo()

{'type': 'Polygon',
 'coordinates': [[[110, 25], [115, 25], [115, 30], [110, 30], [110, 25]]]}

In [65]:
Map.addLayer(region.buffer(10), {'color':'red'}, 'roi')

In [64]:
Map

Map(bottom=14063.0, center=[27.6543381066919, 112.61425887007147], controls=(WidgetControl(options=['position'…

In [33]:
import os
filename = os.path.abspath(sv_file+'.tif')
basename = os.path.basename(filename)
name = os.path.splitext(basename)[0]

In [34]:
name

'TEMP'

In [60]:
crs = 'epsg:4326'
params = {"name": name, "filePerBand": False}
params['scale'] = 60
params["region"] = region
params["crs"] = crs
try:
    url = image.getDownloadURL(params)
    print(url)
except Exception as e:
    print(e)

Total request size (1210723332 bytes) must be less than or equal to 50331648 bytes.


In [49]:
image.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [7, 7],
   'origin': [109, 24],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [7, 7],
   'origin': [109, 24],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [7, 7],
   'origin': [109, 24],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [7, 7],
   'origin': [109, 24],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0

In [26]:
import requests
r = requests.get(url, stream=True, timeout=300, proxies=None)

In [31]:
with open(sv_file+'.zip', "wb") as fd:
    for chunk in r.iter_content(chunk_size=1024):
        fd.write(chunk)

In [ ]:
#clip_dow_merge(region, flooding_freq, freqName, 30)
geemap.ee_export_image_to_drive(freq_image, description= freqName, folder='ee_export', scale=30)
# geemap.ee_export_image_to_drive(flooding_freq.toUint8(), description= freqName, folder='Water', maxPixels=100000000000, region = region, scale=30)
print('download to drive successful!!')